In [1]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier

In [360]:
df = pd.read_csv('data/clean/train_clean_v2.csv')

In [13]:
df_targets = pd.read_csv('data/clean/train_labels.csv')

In [361]:
df.shape, df_test.shape

((10795392, 27), (10795392, 24))

In [362]:
df.head()

,fecha_dato,fecha_dato_year,fecha_dato_month,fecha_dato_day,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,...,indresi,indext,canal_entrada,indfall,tipodom,cod_prov,nomprov,ind_actividad_cliente,renta,segmento
0,2015-01-28,2015,1,28,1375586,1,1,1,35,2015-01-12,...,2,1,20,1,1.0,29.0,7,1.0,87218.10,1
1,2015-01-28,2015,1,28,1050611,1,1,1,23,2012-08-10,...,2,2,1,1,1.0,13.0,23,0.0,35548.74,2
2,2015-01-28,2015,1,28,1050612,1,1,1,23,2012-08-10,...,2,1,1,1,1.0,13.0,23,0.0,122179.11,2
3,2015-01-28,2015,1,28,1050613,1,1,1,22,2012-08-10,...,2,1,7,1,1.0,50.0,6,0.0,119775.54,2
4,2015-01-28,2015,1,28,1050615,1,1,1,23,2012-08-10,...,2,1,1,1,1.0,45.0,16,0.0,22220.04,2


### Prueba con un solo mes

In [854]:
x_train = df[df.fecha_dato == '2015-01-28'].drop(['fecha_dato', 'fecha_alta'], axis=1)
x_test = df[df.fecha_dato == '2015-02-28'].drop(['fecha_dato', 'fecha_alta'], axis=1)

y_train = df_targets.loc[x_train.index, :]
y_test = df_targets.loc[x_test.index, :]

x = x_train.as_matrix()
x_t = x_test.as_matrix()

print('Train Data: {} - targets {}'.format(x_train.shape, y_train.shape))
print('Test Data: {} - targets {}'.format(x_test.shape, y_test.shape))

Train Data: (513791, 25) - targets (513791, 24)
Test Data: (516199, 25) - targets (516199, 24)


Prueba con el tercer producto *`ind_cco_fin_ult1` - Saving Accounts*

In [65]:
y = y_train.as_matrix()[:, 2]

In [650]:
%%time
rf = RandomForestClassifier(n_jobs=4)
rf.fit(x, y)

CPU times: user 18.3 s, sys: 148 ms, total: 18.5 s
Wall time: 5.4 s


In [44]:
%%time
predictions = rf.predict(x_test)

CPU times: user 1.76 s, sys: 36 ms, total: 1.8 s
Wall time: 1.8 s


In [45]:
%%time
rf.score(x_test, y_test[:, 2])

CPU times: user 1.82 s, sys: 48 ms, total: 1.87 s
Wall time: 1.87 s


0.98025955106460883

In [46]:
predictions[:10]

array([ 1.,  0.,  1.,  1.,  0.,  1.,  0.,  1.,  1.,  1.])

In [47]:
pred_probs = rf.predict_proba(x_test)

In [49]:
pred_probs[:10].max(axis=1)

array([ 1. ,  0.7,  0.9,  0.7,  0.8,  0.9,  0.5,  1. ,  0.7,  1. ])

---
### Creación de archivo de submission

In [857]:
cols = [sorted(df_targets.columns.tolist()*2), ['pred', 'prob']*(len(df_targets.columns.tolist()))]

In [858]:
tuples = list(zip(*cols))
index = pd.MultiIndex.from_tuples(tuples)

In [859]:
index

MultiIndex(levels=[['ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1', 'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1', 'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1', 'ind_deco_fin_ult1', 'ind_dela_fin_ult1', 'ind_deme_fin_ult1', 'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1', 'ind_nom_pens_ult1', 'ind_nomina_ult1', 'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1', 'ind_recibo_ult1', 'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1'], ['pred', 'prob']],
           labels=[[0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14, 15, 15, 16, 16, 17, 17, 18, 18, 19, 19, 20, 20, 21, 21, 22, 22, 23, 23], [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]])

In [871]:
result = pd.DataFrame(columns=index)

In [872]:
result.insert(0, 'ncodpers', x_test.ncodpers.values)

In [ ]:
%%time
rf = RandomForestClassifier(n_jobs=4)

for i, col in enumerate(df_targets.columns):
    y = y_train.loc[:, col].as_matrix()
    rf.fit(x, y)
    
    pred_probs = rf.predict_proba(x_t).max(axis=1)
    preds = rf.predict(x_t)
    result[col] = np.vstack((preds, pred_probs)).T

**copy_res:** DataFrame que será el archivo de submission

In [ ]:
copy_res = pd.DataFrame(columns=['ncodpers'] + df_targets.columns.tolist() + ['added_products'])
copy_res['ncodpers'] = result['ncodpers']

In [ ]:
for i in df_targets.columns:
    copy_res[i] = list(zip(*[[i]*20, result[i]['pred'] * result[i]['prob']]))

In [ ]:
copy_res.head()

### Generación de targets para usar la métrica

Diccionario de los productos con valores asignados para poder evaluarlos en la métrica

In [ ]:
prods = {}
for i, col in enumerate(df_targets.columns.tolist()):
    prods[col] = i+1

In [ ]:
prev_tsmonth = "2015-01-28"
test_month = "2015-02-28"

In [ ]:
groupby_fecha_dato = df.groupby(['fecha_dato'])

In [ ]:
ncodpers_prev_tsmonth = groupby_fecha_dato.get_group(prev_tsmonth).ncodpers
ncodpers_test_month = copy_res.ncodpers

In [569]:
"""%%time
for i in copy_res.index:
    values = copy_res.iloc[i, 1:-1].values
    copy_res.loc[i, 'added_products'] = "".join([j[0] for j in sorted(values, key=lambda x: x[1], reverse=True)])"""

CPU times: user 20 ms, sys: 0 ns, total: 20 ms
Wall time: 19.7 ms


### Código para generar el archivo de submission

<h3 style="color:red;">
    Creo que <code>predicted_targ</code> y <code>predicted_prods</code> son iguales. REVISAR
</h3>

* **predicted:** es una lista de listas que continen las predicciones hechas

In [ ]:
#%%time
predicted = []

rs = df_targets.columns.tolist()
for ncodper in ncodpers_test_month.values:
    if ncodper in ncodpers_prev_tsmonth.values:
        
        pre_targ = df_targets.loc[ncodpers_prev_tsmonth[ncodpers_prev_tsmonth == ncodper].index].as_matrix().reshape(24)
        
        predicted_targ = result.iloc[ncodpers_test_month[ncodpers_test_month == ncodper].index, 1:].as_matrix().reshape(48)
        predicted_targ = np.array([predicted_targ[i] for i in range(0, predicted_targ.shape[0], 2)])
        
        line = (pre_targ - predicted_targ) #line es un vector que indica los productos que el cliente ya tiene el mes anterior
      
        #s = " ".join([rs[i] for i, t in enumerate(line) if t == 1])
        #df_result.iloc[i , -1] = " ".join([rs[i] for i, t in enumerate(line) if t == -1])
        
    else:
        line = result.iloc[ncodpers_test_month[ncodpers_test_month == ncodper].index, 1:].as_matrix().reshape(48)
        line = np.array([line[i] for i in range(0, line.shape[0], 2)])
        #s = " ".join([rs[i] for i, t in enumerate(line) if t == 1])
        #df_result.iloc[i , -1] = " ".join([rs[i] for i, t in enumerate(line) if t == 1])
    
    
    predicted_prods = copy_res.iloc[ncodpers_test_month[ncodpers_test_month == ncodper].index, 1:-1].values.reshape(24)
    final_prediction = np.array([predicted_prods[i] for i, p in enumerate(line) if p != -1])
    predicted_str = " ".join([j[0] for j in sorted(final_prediction, key=lambda x: x[1], reverse=True)][:7])
    
    copy_res.loc[ncodpers_test_month[ncodpers_test_month == ncodper].index, 'added_products'] = predicted_str
    
    predicted.append([prods[i] for i in predicted_str.split()])

---
### Código para generar un archivo con los targets reales y poder usar la métrica

* **actual:** es una lista de listas que contienen los productos nuevos que adquirió cada uno de los clientes

In [ ]:
#%%time
actual = []

rs = df_targets.columns.tolist()
for ncodper in ncodpers_test_month.values:
    if ncodper in ncodpers_prev_tsmonth.values:
        
        pre_targ = df_targets.loc[ncodpers_prev_tsmonth[ncodpers_prev_tsmonth == ncodper].index].as_matrix().reshape(24)
        real_targ = df_targets.loc[ncodpers_test_month[ncodpers_test_month == ncodper].index].as_matrix().reshape(24)
        
        line = (pre_targ - real_targ) #line es un vector que indica los productos que el cliente ya tiene el mes anterior
    
        #s = " ".join([rs[i] for i, t in enumerate(line) if t == 1])
        #df_result.iloc[i , -1] = " ".join([rs[i] for i, t in enumerate(line) if t == -1])
        
    else:
        line = result.iloc[ncodpers_test_month[ncodpers_test_month == ncodper].index, 1:].as_matrix().reshape(48)
        line = np.array([line[i] for i in range(0, line.shape[0], 2)])
        #s = " ".join([rs[i] for i, t in enumerate(line) if t == 1])
        #df_result.iloc[i , -1] = " ".join([rs[i] for i, t in enumerate(line) if t == 1])
    
    s = " ".join([rs[i] for i, t in enumerate(line) if t == 1])
    
    if not s:
        actual.append([])
    else:
        actual.append([prods[i] for i in s.split()])

In [853]:
mapk(actual, predicted, 7)

0.37202380952380948

---
### Métrica de desempeño
Para hacer mediciones locales

In [578]:
"""
From github: @benhamner
"""

def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])